In [1]:
import sys
sys.path.append('../')

import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from utility.file_utility import FileUtility
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, precision_score, recall_score
from gensim.models.wrappers import FastText
import itertools
import os

Using TensorFlow backend.


In [ ]:



class DNNMutliclass16S(object):
    
    def __init__(self, X,Y, model_arch=[500]):
        # rep. X
        self.X=X
        # encoding Y
        self.Y=Y
        self.C=len(set(Y))
        encoder = LabelEncoder()
        encoder.fit(Y)
        self.encoded_Y = encoder.transform(Y)
        self.onehot_y = np_utils.to_categorical(encoded_Y)
        # model's arch
        self.model_arch=model_arch
    
    def get_MLP_model(self):
        # creating the model
        model = Sequential()
        for layer_idx, h_layer_size in self.model_arch:
            if layer_idx==0:
                model.add(Dense(h_layer_size, input_dim=self.X.shape[1], activation='relu'))
            else:
                if h_layer_size < 1:
                    model.add(Dropout(h_layer_size))
                else:
                    model.add(Dense(h_layer_size, activation='relu'))
        model.add(Dense(self.C, activation='softmax'))        
        # Compile model
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model
    
    def cross_validation(self, result_filename, n_fold=5, epochs=50, batch_size=100, model_strct='mlp'):

        skf = StratifiedKFold(n_splits=n_fold, shuffle=True)

        p_micro=[]
        p_macro=[]
        r_micro=[]
        r_macro=[]
        f1_micro=[]
        f1_macro=[]

        for train_index, valid_index in skf.split(X, Y):
            print ('\n Evaluation on a new fold is now get started ..')
            X_train=X[train_index,:]
            y_train=onehot_y[train_index,:]
            y_class_train=encoded_Y[train_index]
            X_valid=X[valid_index,:]
            y_valid=onehot_y[valid_index,:]
            y_class_valid=encoded_Y[valid_index]
            
            if model_strct=='mlp'
                model=self.get_MLP_model()
            
            # fitting
            history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,shuffle=True, validation_data=(X_valid, y_valid), verbose=0)
            pred=model.predict_classes(X_valid)
            # score-calculations
            f1_micro.append(f1_score(pred, y_class_valid, average='micro'))
            f1_macro.append(f1_score(pred, y_class_valid, average='macro'))
            p_micro.append(precision_score(pred, y_class_valid, average='micro'))
            p_macro.append(precision_score(pred, y_class_valid, average='macro'))
            r_micro.append(recall_score(pred, y_class_valid, average='micro'))
            r_macro.append(recall_score(pred, y_class_valid, average='macro'))

        # mean values
        f1mac=np.mean(f1_macro)
        f1mic=np.mean(f1_micro)
        prmac=np.mean(p_macro)
        prmic=np.mean(p_micro)
        remac=np.mean(r_macro)
        remic=np.mean(r_micro)
        # std values
        sf1mac=np.std(f1_macro)
        sf1mic=np.std(f1_micro)
        sprmac=np.std(p_macro)
        sprmic=np.std(p_micro)
        sremac=np.std(r_macro)
        sremic=np.std(r_micro)
        # table
        latex_line=' & '.join([str(np.round(x,2))+' $\\pm$ '+str(np.round(y,2)) for x,y in [[prmic, sprmic], [remic, sremic], [f1mic, sf1mic], [prmac, sprmac], [remac, sremac], [f1mac, sf1mac] ]])      
        
        FileUtility.save_obj('_'.join([filename, model_strct,'-'.join([str(x) for x in self.model_arch]), str(f1mac)]), [latex_line, p_micro, r_micro, f1_micro, p_macro, r_macro, f1_macro, history])
        
        FileUtility.save_obj('_'.join([filename, 'layers', model_strct,'-'.join([str(x) for x in self.model_arch]), str(f1mac)]), model.layers)
        


In [ ]:
'../../datasets/results/eco/classifier/nn'